In [ ]:
!pip install fuzzywuzzy

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import pandas as pd
import uuid
from google.colab import drive

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
CULINARYDB_RECIPE_DETAILS = '/content/drive/Shareddrives/Projeto BD - MC536/CulinaryDB/01_Recipe_Details.csv'
CULINARYDB_INGREDIENTS = '/content/drive/Shareddrives/Projeto BD - MC536/CulinaryDB/02_Ingredients.csv'
CULINARYDB_COMPOUND_INGREDIENTS = '/content/drive/Shareddrives/Projeto BD - MC536/CulinaryDB/03_Compound_Ingredients.csv'
CULINARYDB_RECIPE_INGREDIENTS = '/content/drive/Shareddrives/Projeto BD - MC536/CulinaryDB/04_Recipe-Ingredients_Aliases.csv'

FOODB_FOOD = '/content/drive/Shareddrives/Projeto BD - MC536/FoodDB/FooDB/Food.csv'
FOODB_CONTENT = '/content/drive/Shareddrives/Projeto BD - MC536/FoodDB/FooDB/Content.csv'
FOODB_NUTRIENT = '/content/drive/Shareddrives/Projeto BD - MC536/FoodDB/FooDB/Nutrient.csv'
FOODB_COMPOUND = '/content/drive/Shareddrives/Projeto BD - MC536/FoodDB/FooDB/Compound.csv'

In [ ]:
culinarydb_recipe_details = pd.read_csv(CULINARYDB_RECIPE_DETAILS)
culinarydb_ingredients = pd.read_csv(CULINARYDB_INGREDIENTS)
culinarydb_compound_ingredients = pd.read_csv(CULINARYDB_COMPOUND_INGREDIENTS)
culinarydb_recipe_ingredients = pd.read_csv(CULINARYDB_RECIPE_INGREDIENTS)

foodb_food = pd.read_csv(FOODB_FOOD)
foodb_content = pd.read_csv(FOODB_CONTENT)
foodb_nutrient = pd.read_csv(FOODB_NUTRIENT)
foodb_compound = pd.read_csv(FOODB_COMPOUND)

<ipython-input-5-70f5d42c0705>:7: DtypeWarning: Columns (4,5,6,7,8,9,13,14,21,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  foodb_content = pd.read_csv(FOODB_CONTENT)
<ipython-input-5-70f5d42c0705>:9: DtypeWarning: Columns (3,4,5,6,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  foodb_compound = pd.read_csv(FOODB_COMPOUND)


In [ ]:
culinarydb_aliased_ingredient_name = culinarydb_ingredients['Aliased Ingredient Name'].tolist()
foodb_name = foodb_food['name'].tolist()

# Create an empty dictionary to store the matches
matches = {}

# Loop through the culinary ingredients
for ingredient in culinarydb_aliased_ingredient_name:
    # Use fuzzywuzzy to find the closest match in the foodb ingredients
    match, score = process.extractOne(ingredient, foodb_name)

    # You might want to set a threshold for the score to make sure the matches are accurate
    if score > 70 and abs(len(ingredient) - len(match)) < 3:
        matches[ingredient] = match

# Criando dataframe de comidas


In [ ]:
rows = []

for ingredient_name in matches:
    foodb_ingredient_name = matches[ingredient_name]
    foodb_ingredient = foodb_food[foodb_food['name'] == foodb_ingredient_name].iloc[0]
    culinarydb_ingredient = culinarydb_ingredients[culinarydb_ingredients['Aliased Ingredient Name'] == ingredient_name].iloc[0]

    row = {
        'id': uuid.uuid1(),
        'name': foodb_ingredient_name,
        'name_scientific': foodb_ingredient['name_scientific'],
        'wikipedia_id': foodb_ingredient['wikipedia_id'],
        'food_group': foodb_ingredient['food_group'],
        'food_subgroup': foodb_ingredient['food_subgroup'],
        'food_type': foodb_ingredient['food_type'],
        'culinarydb_ingredient_id': culinarydb_ingredient['Entity ID'],
        'foodb_id': foodb_ingredient['id']
    }
    rows.append(row)

sexto_ingredient = pd.DataFrame(rows)

In [ ]:
foodb_ids = sexto_ingredient['foodb_id']

filtered_foodb_food = foodb_food[foodb_food['id'].isin(foodb_ids)]
filtered_foodb_nutrient = foodb_nutrient[foodb_nutrient['id'].isin(foodb_ids)]
filtered_foodb_compound = foodb_compound[foodb_compound['id'].isin(foodb_ids)]

filtered_foodb_content = foodb_content[foodb_content['food_id'].isin(foodb_ids)].dropna(subset=['orig_content'])

filtered_foodb_content_nutrient = filtered_foodb_content.loc[filtered_foodb_content['source_type'] == 'Nutrient']
filtered_foodb_content_compound = filtered_foodb_content.loc[filtered_foodb_content['source_type'] == 'Compound']

# make sure content compound has compounds that exists
mask = filtered_foodb_content_compound['source_id'].isin(filtered_foodb_compound['id'])
filtered_foodb_content_compound = filtered_foodb_content_compound[mask]

# make sure content nutrient has nutrient that exists
mask = filtered_foodb_content_nutrient['source_id'].isin(filtered_foodb_nutrient['id'])
filtered_foodb_content_nutrient = filtered_foodb_content_nutrient[mask]

In [ ]:
culinarydb_ids = sexto_ingredient['culinarydb_ingredient_id']

filtered_culinarydb_ingredients = culinarydb_ingredients[culinarydb_ingredients['Entity ID'].isin(culinarydb_ids)]
filtered_culinarydb_compound_ingredients = culinarydb_compound_ingredients[culinarydb_compound_ingredients['entity_id'].isin(culinarydb_ids)]
filtered_culinarydb_recipe_ingredients = culinarydb_recipe_ingredients[culinarydb_recipe_ingredients['Entity ID'].isin(culinarydb_ids)]

culinarydb_recipe_ids = filtered_culinarydb_recipe_ingredients['Recipe ID']

filtered_culinarydb_recipe_details = culinarydb_recipe_details[culinarydb_recipe_details['Recipe ID'].isin(culinarydb_recipe_ids)]

# rename recipes columns
filtered_culinarydb_recipe_ingredients = filtered_culinarydb_recipe_ingredients.rename(columns={
    'Recipe ID': 'recipe_id',
    'Original Ingredient Name': 'original_ingredient',
    'Aliased Ingredient Name': 'aliased_ingredient',
    'Entity ID': 'culinarydb_ingredient_id',
})

filtered_culinarydb_recipe_details = filtered_culinarydb_recipe_details.rename(columns={
    'Recipe ID': 'recipe_id',
})

In [ ]:
filtered_foodb_food.to_csv('/content/drive/Shareddrives/Projeto BD - MC536/filtrado/filtered_foodb_food.csv', index=False)
filtered_foodb_nutrient.to_csv('/content/drive/Shareddrives/Projeto BD - MC536/filtrado/filtered_foodb_nutrient.csv', index=False)
filtered_foodb_compound.to_csv('/content/drive/Shareddrives/Projeto BD - MC536/filtrado/filtered_foodb_compound.csv', index=False)
filtered_foodb_content.to_csv('/content/drive/Shareddrives/Projeto BD - MC536/filtrado/filtered_foodb_content.csv', index=False)
filtered_foodb_content_nutrient.to_csv('/content/drive/Shareddrives/Projeto BD - MC536/filtrado/filtered_foodb_content_nutrient.csv', index=False)
filtered_foodb_content_compound.to_csv('/content/drive/Shareddrives/Projeto BD - MC536/filtrado/filtered_foodb_content_compound.csv', index=False)

filtered_culinarydb_ingredients.to_csv('/content/drive/Shareddrives/Projeto BD - MC536/filtrado/filtered_culinarydb_ingredients.csv', index=False)
filtered_culinarydb_compound_ingredients.to_csv('/content/drive/Shareddrives/Projeto BD - MC536/filtrado/filtered_culinarydb_compound_ingredients.csv', index=False)
filtered_culinarydb_recipe_ingredients.to_csv('/content/drive/Shareddrives/Projeto BD - MC536/filtrado/filtered_culinarydb_recipe_ingredients.csv', index=False)
filtered_culinarydb_recipe_details.to_csv('/content/drive/Shareddrives/Projeto BD - MC536/filtrado/filtered_culinarydb_recipe_details.csv', index=False)

sexto_ingredient.to_csv('/content/drive/Shareddrives/Projeto BD - MC536/filtrado/sexto_ingredient.csv', index=False)

In [ ]:
def create_row(foodb_content_compound):
    ingredient = sexto_ingredient[sexto_ingredient['foodb_id'] == foodb_content_compound['food_id']].iloc[0]
    return {
        'id': uuid.uuid1(),
        'compound_id': foodb_content_compound.source_id,
        'food_id': ingredient.id,
        'orig_source_id': foodb_content_compound.orig_source_id,
        'orig_content': foodb_content_compound.orig_content,
        'orig_unit': foodb_content_compound.orig_unit,
        'preparation_type': foodb_content_compound.preparation_type,
        'standard_content': foodb_content_compound.standard_content
    }

sexto_content_compound = filtered_foodb_content_compound.apply(create_row, axis=1, result_type='expand')

In [ ]:
sexto_content_compound.to_csv('/content/drive/Shareddrives/Projeto BD - MC536/filtrado/sexto_content_compound.csv', index=False)

In [ ]:
culinarydb_cuisines = pd.DataFrame(filtered_culinarydb_recipe_details['Cuisine'].drop_duplicates())
culinarydb_cuisines.to_csv('/content/drive/Shareddrives/Projeto BD - MC536/filtrado/culinarydb_cuisines.csv', index=False)

In [ ]:
def create_row(foodb_content_nutrient):
    ingredient = sexto_ingredient[sexto_ingredient['foodb_id'] == foodb_content_nutrient['food_id']].iloc[0]
    return {
        'id': uuid.uuid1(),
        'nutrient_id': foodb_content_nutrient.source_id,
        'food_id': ingredient.id,
        'orig_source_id': foodb_content_nutrient.orig_source_id,
        'orig_content': foodb_content_nutrient.orig_content,
        'orig_unit': foodb_content_nutrient.orig_unit,
        'preparation_type': foodb_content_nutrient.preparation_type,
        'standard_content': foodb_content_nutrient.standard_content
    }

sexto_content_nutrient = filtered_foodb_content_nutrient.apply(create_row, axis=1, result_type='expand')

In [ ]:
sexto_content_nutrient.to_csv('/content/drive/Shareddrives/Projeto BD - MC536/filtrado/sexto_content_nutrient.csv', index=False)

In [ ]:
filtered_foodb_content_nutrient.to_csv('/content/drive/Shareddrives/Projeto BD - MC536/filtrado/filtered_foodb_content_nutrient.csv', index=False)

In [ ]:
filtered_culinarydb_recipe_details.iloc[0]

recipe_id                               1
Title        5 spice vegetable fried rice
Source                        TARLA_DALAL
Cuisine               Indian Subcontinent
Name: 0, dtype: object